# <font color=green> Capstone Project - Applied Data Science by IBM <font>
## Choice of Italian or Indian style restaurant in Clermont -Ferrand France

## Table of contents
### Introduction
### Data
### Analysis
### Result and Discusion
### Conclusion

In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
!conda install -c conda-forge geopy --yes # 
#import geocoder # import geocoder
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
address = 'Place de Jaude, Clermont-Ferrand, FR'

geolocator = Nominatim(user_agent="fr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Place de Jaude is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Place de Jaude is 45.77641275, 3.08213651373141.


In [3]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math


     |████████████████████████████████| 1.5MB 17.4MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 17.8MB/s eta 0:00:01


In [4]:

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=31, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=31, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('city center longitude={}, latitude={}'.format(longitude, latitude))
x, y = lonlat_to_xy(longitude, latitude)
print('City center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('City center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
city center longitude=3.08213651373141, latitude=45.77641275
City center UTM X=506385.5888463631, Y=5069209.183415899
City center longitude=3.08213651373141, latitude=45.776412749999984


In [6]:
# radius of 200 metres. Neighborhoods would be 200 meters apart and 1 km from the city center
city_center_x, city_center_y = lonlat_to_xy(longitude, latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
print (k)
x_min = city_center_x - 500
x_step = 200
y_min = city_center_y - 1500 #- (int(21/k)*k*200 - 8000)/2

y_step = 200 * k 

jaude_lat = []
jaude_lng = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 20 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(city_center_x, city_center_y, x, y)
        if (distance_from_center <= 1001):
            lon, lat = xy_to_lonlat(x, y)
            jaude_lat.append(lat)
            jaude_lng.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(jaude_lat), 'candidate neighborhood centers generated.')

0.8660254037844386
79 candidate neighborhood centers generated.


In [7]:
map_jaude = folium.Map(location=[latitude,longitude], zoom_start=16)
folium.Marker([latitude,longitude], popup='Place de Jaude').add_to(map_jaude)
for lat, lon in zip(jaude_lat, jaude_lng):
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_jaude) 
    #folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_jaude)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_jaude

### OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~1km from Place de Jaude.

In [ ]:
#reverse coding to obtain address from co ordinates
#list to store the addresses obtained. 
addresses = []
for lat, lon in zip(jaude_lat, jaude_lng):
    geolocator = Nominatim(user_agent="fr_explorer")
    location = geolocator.reverse([lat, lon],addressdetails=False)
    #location = location.replace(', France', '') #  remove country part of the address
    if location is None:
        location = 'NO ADDRESS'    
    addresses.append(location)
    

In [9]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': jaude_lat,
                             'Longitude': jaude_lng,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

#df_locations['Start']=df_locations['Address'].str.find(",C")
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"(55, Boulevard Jean Jaurès, Les Salins, Clermo...",45.767590,3.080837,506285.588846,5.068229e+06,985.471599
1,"(3, Rue du Tonnet, Les Salins, Clermont-Ferran...",45.767588,3.083410,506485.588846,5.068229e+06,985.471599
2,"(Berlioz, Rue Berlioz, Chamalières, Clermont-F...",45.769152,3.075953,505905.588846,5.068402e+06,939.116090
3,"(Rue de Vallières, Les Salins, Clermont-Ferran...",45.769150,3.078525,506105.588846,5.068402e+06,854.364694
4,"(32, Avenue Marx Dormoy, Les Salins, Clermont-...",45.769148,3.081097,506305.588846,5.068402e+06,811.134410
5,"(Coubertin, Place Pierre de Coubertin, Les Sal...",45.769146,3.083669,506505.588846,5.068402e+06,816.050875
6,"(Résidence Pierre de Coubertin, 18, Rue Abbé d...",45.769145,3.086241,506705.588846,5.068402e+06,868.296626
7,"(25, Rue Vermenouze, Saint-Jacques, Clermont-F...",45.769143,3.088814,506905.588846,5.068402e+06,960.176562
8,"(62, Boulevard Aristide Briand, Chamalières, C...",45.770711,3.075697,505885.588846,5.068575e+06,807.417976
9,"(Regensburg, Rue de La Rochefoucault, Les Sali...",45.770709,3.078270,506085.588846,5.068575e+06,701.372789


In [10]:
# save data to csv file
df_locations.to_csv(r'./locations.csv')

In [11]:
#save the data to a file
df_locations.to_pickle('./locations.pkl') 

In [12]:
CLIENT_ID = '1IFAM5PZF250GJTAHHTBTLP2FDYGUKIRWVMRUXA4NWSNIMNP' 
CLIENT_SECRET = 'YVG5UEPH1FIJYIMTCEC4HL1FE5NGUFOXLNQ3FGIWTPN4JV3V' 
VERSION = '20190927' # Foursquare API version

### FourSquare API

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffee shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' and 'India restaurant category, as we need info on Italian and Indian  restaurants in the neighborhood.
We will explore venues within 600m radius from the neighborhood


In [13]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

indian_restaurant_categories = ['4bf58dd8d48988d10f941735','54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dfe8',
                                '54135bf5e4b08f3d2429dff1','54135bf5e4b08f3d2429dff0','54135bf5e4b08f3d2429dfde']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(',Clermont-Ferrand, Francais', '')
    address = address.replace(',Clermont-Ferrand, France', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20190927'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

## We will now get data about all restaurants and italian restaurants from the neighbourhood locations that we have obtained in earlier steps

In [14]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=500 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=500, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            is_res, is_indian = is_restaurant(venue_categories, specific_filter=indian_restaurant_categories)
            #print(is_res) 
            if is_res:
                #print(is_res)
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian,is_indian, x, y)
                if venue_distance<=500:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_500.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_500.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_500.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(jaude_lat, jaude_lng)
    
    # Let's persists this in local file system
    with open('restaurants_500.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_500.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_500.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Restaurant data loaded.


In [15]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 52
Total number of Italian restaurants: 5
Percentage of Italian restaurants: 9.62%
Average number of restaurants in neighborhood: 8.924050632911392


## We will repeat the same process for obtaining Indian restaurants in the neighbourhood locations

In [19]:
import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    indian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=500, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_indian = is_restaurant(venue_categories, specific_filter=indian_restaurant_categories)
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance,is_italian, is_indian, x, y)
                if venue_distance<=500:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_indian:
                    indian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, indian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
indian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_500.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('indian_restaurants_500.pkl', 'rb') as f:
        indian_restaurants = pickle.load(f)
    with open('location_restaurants_500.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, indian_restaurants, location_restaurants = get_restaurants(jaude_lat, jaude_lng)
    
    # Let's persists this in local file system
    with open('restaurants_500.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('indian_restaurants_500.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_500.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [20]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Indian restaurants:', len(indian_restaurants))
print('Percentage of Indian restaurants: {:.2f}%'.format(len(indian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 53
Total number of Indian restaurants: 1
Percentage of Indian restaurants: 1.89%
Average number of restaurants in neighborhood: 9.0


In [21]:
# print list of all restaurants
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4e27feaa1f6e88a1545d3d4f', "De L'eau à la Bouche", 45.771427, 3.083565, 'France', 476, False, False, 506497.22345323395, 5068655.36950919)
('4d4da41f819554813022da93', 'Brasserie du Jardin', 45.7708275104889, 3.0871489366727873, '55 boulevard François Mitterrand, 63000 Clermont-Ferrand, France', 227, False, False, 506775.94903479517, 5068589.062259003)
('52ee2e6b498e71d257936b27', "Au p'tit marmiton", 45.76938533659876, 3.072494502934678, 'France', 418, False, False, 505636.6920506263, 5068427.695895185)
('4fed8560e4b072533f1b687e', 'alfred', 45.7727826335402, 3.081596226069066, 'Rue du Puits Artesien (Rue Lagarlaye), 63000 Clermont-Ferrand, France', 472, False, False, 506343.9965299406, 5068805.825563097)
('4e2ffda018a8ea95b0ab7177', "De la Bouche à l'Oreille", 45.772123641119215, 3.0836919310993762, 'France', 387, False, False, 506507.01140169107, 5068732.778329743)
('4e5f0d78d4c08cf7f5967bec', 'Restaurant Le Gergovia', 45.77040824366

In [22]:
# print list of Italian restaurants
print('List of Italian restaurants')
print('---------------------------')
for i in list(italian_restaurants.values())[:10]:
    print(i)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4c9c7e8dd3c2b60cd8ecb7bc', 'Un Assaggio', 45.773966414060226, 3.0769536428780477, 'Clermont-Ferrand, France', 448, True, False, 505982.91537236376, 5068936.988580587)
('4f3ab173e4b05505712b7b41', 'N°3', 45.77378874448363, 3.0850155035131723, 'Rue Georges Clemenceau, 63000 Clermont-Ferrand, France', 446, True, False, 506609.7218810226, 5068917.883908532)
('4d25b0e6bcc1224b1be5921b', 'Pizzeria Italia', 45.77513116580566, 3.083911291825762, "mail d'Allagnat (5 place Louis Aragon), 63000 Clermont-Ferrand, France", 416, True, False, 506523.7159306328, 5069066.939517381)
('4cb0de4039458cfa8acd06a0', 'Bistro Vénitien', 45.77871170588981, 3.0837842263383792, '32 rue des Gras, 63000 Clermont-Ferrand, France', 497, True, False, 506513.4202989816, 5069464.736105854)
('5179784ee4b0b9ec8469f127', 'Il Visconti', 45.77858645389464, 3.087191235264401, 'France', 469, True, False, 506778.2977487672, 5069451.103502762)
...
Total: 5


In [23]:
# print restaurants around location
print('Restaurants around location')
print('---------------------------')
for i in range(50, 79):
    rs = location_restaurants[i][:10]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 51: Les Arcandiers, Le Sisisi, Il Visconti, Le Boeuf Café, Le Puy de la Lune, Bistro Vénitien, La Gourmandine, Café Bui, Los Dos Hermanos, Pizzeria Italia
Restaurants around location 52: Les Arcandiers, Le Sisisi, Il Visconti, Le Boeuf Café, Café Bui, Bistro Vénitien, Taverne de Maître Kanter, Le 1513, Konnichiwa, Mai-Lan
Restaurants around location 53: Les Arcandiers, Il Visconti, Le Sisisi, La Tente Berbère, Taverne de Maître Kanter, Mai-Lan, Avenue*, Le Choucas
Restaurants around location 54: La Tente Berbère, Mai-Lan, Kebab de la gare, Le Choucas
Restaurants around location 55: Le Puy de la Lune, La Gourmandine, Santooka, Le Petit Chaperon Rouge, Okinawa, Lard et la Manière, A La Bonne Heure, Le Bouchon de Jaude
Restaurants around location 56: Le Puy de la Lune, La Gourmandine, Le Boeuf Café, Santooka, Café Bui, Bistro Vénitien, Okinawa, Le Petit Chaperon Rouge, Lard et la Manière, Le Faisan Doré
Re

In [115]:
len(location_restaurants)

79

#### We will mark all the restaurants obtained in our area on the map. The Italian restaurants will be marked color coded as Green and Indian restaurants will be marked color coded as Red. 

In [24]:
map_jaude = folium.Map(location=[latitude,longitude], zoom_start=16)
folium.Marker([latitude,longitude], popup='Place de Jaude').add_to(map_jaude)
for key,value in restaurants.items():
    lat = value[2]; lon = value[3]
    #print(value)
    is_italian = value[6]
    is_indian = value[7]
    #print(is_italian)
    if is_italian: 
        color = 'green' 
    elif is_indian:
        color ='red'  
    else :
        color = 'blue'
    folium.CircleMarker([lat, lon], radius=4, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_jaude) 
    #folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_jaude)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_jaude

In the above steps we have gathered data about all the restaurants within 1 km of the city centre, Place de Jaude. We have also identified the Italian and Indian restaurants amongst the list of restaurants obtained in the neighborhood. We also know which restaurants are situated in the vicinity of the candidate neighborhood. 
We have explored the restaurants within the area of 500 mts from the neoighborhood so that we attemt to get a full coverage do not miss any restaurants in the vicinity. We have therefore narrowed it down to a total of 52 restaurants with 5 Italian restaurants and 1 Indian restaurant amongst the candidat neighborhood
The data gathering phase concludes here. We will use this data gathered for further analysis and report.

## Methodology

In this project we will direct our efforts to find areas in Clermont Ferrand with low restaurant density and also areas with less Italian and/or India restaurants. We will limit our area of exploration and analysis to within 1 km of the city center, Place de Jaude.
In the first step we have gathered data about all the restaurants within 1 km of the city centre, Place de Jaude. We have also identified the Italian and Indian restaurants amongst the list of restaurants obtained in the neighborhood. We also know which restaurants are situated in the vicinity of the candidate neighborhood. 
Our second step will be the analysis of the data so collected and narrowed down. In this step we will do some exploratory analysis of the restaurant density of the neighborhood area of Clermont -Ferrand. We will use **Heat maps** to identify promising areas with low restaurant density and also no *Italian or Indian* restaurant in the vicinity. We can then focus on these arears so highlighted

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Italian and/or Indian restaurants in radius of 300 meters. 
We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.


## Analysis
Let us perform some basic exploratory analysis. We will start by taking a count of the restaurants in the candidate neighborhoods identified

In [25]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=500m:', np.array(location_restaurants_count).mean())

df_locations.head(10)
df_locations.to_csv(r'./locations.csv')

Average number of restaurants in every area with radius=500m: 9.0


In [26]:
# store the restaurants and specific style restaurant list in a dataframe and csv file. 
#df_rest_500 = pd.DataFrame(restaurants{})
#df_italian_500 = pd.DataFrame([italian_restaurants])
#df_indian_500 = pd.DataFrame([indian_restaurants])

Now we will calculate the distance to nearest Italian and/or India restaurant from every neighborhood area.

In [32]:
# distance to Italian restaurant
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 5000
    for res in italian_restaurants.values():
        res_x = res[8]
        res_y = res[9]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

# distance to India restaurant
distances_to_indian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 5000
    for res in indian_restaurants.values():
        res_x = res[8]
        res_y = res[9]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_indian_restaurant.append(min_distance)

df_locations['Distance to Indian restaurant'] = distances_to_indian_restaurant

In [33]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant,Distance to Indian restaurant
0,"(55, Boulevard Jean Jaurès, Les Salins, Clermo...",45.767590,3.080837,506285.588846,5.068229e+06,985.471599,1,761.512118,1490.574736
1,"(3, Rue du Tonnet, Les Salins, Clermont-Ferran...",45.767588,3.083410,506485.588846,5.068229e+06,985.471599,1,700.176758,1536.820627
2,"(Berlioz, Rue Berlioz, Chamalières, Clermont-F...",45.769152,3.075953,505905.588846,5.068402e+06,939.116090,1,540.544330,1302.754416
3,"(Rue de Vallières, Les Salins, Clermont-Ferran...",45.769150,3.078525,506105.588846,5.068402e+06,854.364694,2,548.869349,1298.142390
4,"(32, Avenue Marx Dormoy, Les Salins, Clermont-...",45.769148,3.081097,506305.588846,5.068402e+06,811.134410,5,598.856621,1324.076381
5,"(Coubertin, Place Pierre de Coubertin, Les Sal...",45.769146,3.083669,506505.588846,5.068402e+06,816.050875,1,526.285130,1378.833877
6,"(Résidence Pierre de Coubertin, 18, Rue Abbé d...",45.769145,3.086241,506705.588846,5.068402e+06,868.296626,0,524.712135,1459.173553
7,"(25, Rue Vermenouze, Saint-Jacques, Clermont-F...",45.769143,3.088814,506905.588846,5.068402e+06,960.176562,2,594.701278,1561.150875
8,"(62, Boulevard Aristide Briand, Chamalières, C...",45.770711,3.075697,505885.588846,5.068575e+06,807.417976,2,374.642560,1133.019580
9,"(Regensburg, Rue de La Rochefoucault, Les Sali...",45.770709,3.078270,506085.588846,5.068575e+06,701.372789,3,376.067065,1124.161005


In [34]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())
print('Average distance to closest Indian restaurant from each area center:', df_locations['Distance to Indian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 394.3180313686757
Average distance to closest Indian restaurant from each area center: 874.7777440275822


So on an average an **Italian restaurant** can be found **within 400 meters** from each neighbourhood area and an **Indian restaurant** can be **found within 900** meters of the neighbourhood area.

Let's create a **Heatmap** showing the **density** of the restaurants. We can try to obtain more meaningful information from the map.
Let us show a few markers indicating distance of 0.5 km, 1 km and 1.5 km from Place de Jaude

In [36]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]
italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]
indian_latlons = [[res[2],res[3]] for res in indian_restaurants.values()]

In [38]:
from folium import plugins
from folium.plugins import HeatMap

map_jaude = folium.Map(location=[latitude,longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_jaude)
folium.Marker([latitude,longitude]).add_to(map_jaude)
folium.Circle([latitude,longitude], radius=500, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1000, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1500, fill=False, color='white').add_to(map_jaude)

map_jaude

Looks like some pockets of low density restaurant areas can be found to the west, south west and north west of the city centre.

Let us make a **Heat map** showing the **density** of the *Italian restaurant* and *Indian Restaurant* only.

## Italian Restaurant density

In [39]:
map_jaude = folium.Map(location=[latitude,longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_jaude)
folium.Marker([latitude,longitude]).add_to(map_jaude)
folium.Circle([latitude,longitude], radius=500, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1000, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1500, fill=False, color='white').add_to(map_jaude)

map_jaude

The map is not so 'Hot'. But we do have some density of Italian restaurants to the North east and south east of the city centre.

## Indian Restaurant density

In [40]:
map_jaude = folium.Map(location=[latitude,longitude], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude) #cartodbpositron cartodbdark_matter
HeatMap(indian_latlons).add_to(map_jaude)
folium.Marker([latitude,longitude]).add_to(map_jaude)
folium.Circle([latitude,longitude], radius=500, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1000, fill=False, color='white').add_to(map_jaude)
folium.Circle([latitude,longitude], radius=1500, fill=False, color='white').add_to(map_jaude)

map_jaude

In case of the India restaurant category we see that the map is pretty cool. There is no visible density of restaurants close to the city center and also within 1 km from the city centre. There is just 1 restaurant in the Fontgieve quarters which is about 700 meters from the city centre.

We can focus our attention to the Bonnabaud quartier of the city. This area is not far from the city center. It is about 500 meters from Place de Jaude. <br>
We also have a low density of restaurants in this area. The total presence of Italian restaurants is only 9% and for India restaurants it is just 1% of the total number of restaurants.<br>

Analysis of the map created and also of websites relating to the Bonnabaud areas shows us that Rue Blatin and Avenue Julien are two of the main streets in this area leading to the cut center. These streets are lined with residences as well as shopping areas and flagship shops leading to the city centre.<br>

In the below steps we will define new candidate area which would be around Bonnabaud. For the new ares we would keep the neighbourhoods closer, maybe 50mts apart and 500 meters from the city centre.


In [51]:
roi_x_min = city_center_x - 300
roi_y_max = city_center_y + 60
roi_width =500
roi_height = 500
roi_center_x = roi_x_min 
roi_center_y = roi_y_max 
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]
print(roi_center)

map_jaude = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_jaude)
folium.Marker(roi_center).add_to(map_jaude)
folium.Circle(roi_center, radius=500, fill=False, color='white').add_to(map_jaude)

map_jaude

[45.77695550017487, 3.078278434878929]


Let us create the new grid locations for the region of interest determined. We will obtain neighbourhood areas that are closer to each other. That is we will place the neighbourhoods 200 mts apart from each other.

In [86]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 200
y_step = 200 * k 
roi_x_min = roi_center_x -300
roi_y_min = roi_center_y - 500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 1001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

57 candidate neighborhood centers generated.


In [87]:
# map the neighbourhood areas
map_jaude = folium.Map(location=[latitude,longitude], zoom_start=16)
folium.Marker([latitude,longitude], popup='Place de Jaude').add_to(map_jaude)
for lat, lon in zip(roi_latitudes, roi_longitudes):
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_jaude) 
map_jaude

Now we will calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Italian and/or Indian restaurant**.

In [89]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[8]; res_y = res[9]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 10000
    for res in restaurants.values():
        res_x = res[8]; res_y = res[9]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_italian_distances = []
roi_indian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, italian_restaurants)
    roi_italian_distances.append(distance)
    distance_i = find_nearest_restaurant(x, y, indian_restaurants)
    roi_indian_distances.append(distance_i)

print('done.')


Generating data on location candidates... done.


In [90]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Italian restaurant':roi_italian_distances,
                                 'Distance to Indian restaurant': roi_indian_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Italian restaurant,Distance to Indian restaurant
0,45.772457,3.075057,505835.588846,5.068769e+06,1,223.301766,950.354038
1,45.772456,3.077629,506035.588846,5.068769e+06,1,175.877992,929.073407
2,45.772454,3.080201,506235.588846,5.068769e+06,3,303.319069,950.358876
3,45.772452,3.082774,506435.588846,5.068769e+06,7,228.985044,1011.526861
4,45.772450,3.085346,506635.588846,5.068769e+06,5,150.933550,1105.979741
5,45.772448,3.087919,506835.588846,5.068769e+06,3,270.421380,1226.048851
6,45.774017,3.074416,505785.588846,5.068942e+06,1,197.400398,796.135152
7,45.774015,3.076988,505985.588846,5.068942e+06,1,6.025492,757.519489
8,45.774013,3.079561,506185.588846,5.068942e+06,6,202.745397,770.610391
9,45.774011,3.082133,506385.588846,5.068942e+06,11,185.989377,832.973572


In [91]:
# write to csv file for any further use for the data
df_roi_locations.to_csv(r'./roi_locations.csv')

### Now we need to filter this data as per our specifications: the area should have not more than 3 restaurants and no Italian/Indian restaurant in the radius of 300 mts

In [99]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=3))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Italian restaurant']>=300)
print('Locations with no Italian restaurants within 300m:', good_ita_distance.sum())

good_ind_distance = np.array(df_roi_locations['Distance to Italian restaurant']>=300)
print('Locations with no Indian restaurants within 300m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance,good_ind_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 34
Locations with no Italian restaurants within 500m: 28
Locations with no Indian restaurants within 500m: 28
Locations with both conditions met: 24


In [96]:
df_good_locations.to_csv(r'./good_locations.csv')

#### We will plot the locations on the map

In [151]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_jaude = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude)
HeatMap(restaurant_latlons).add_to(map_jaude)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.3).add_to(map_jaude)
folium.Marker([latitude,longitude]).add_to(map_jaude)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=4, color='green', fill=True, fill_color='green', fill_opacity=1).add_to(map_jaude) 

map_jaude

We now have a bunch of locations fairly close to Place de Jaude ( in Bonnabaud and Fontgieve quartiers), and we know that each of those locations has no more than three restaurants in radius of 250m, and no Italian and/or Indian restaurant closer than 300m. Any of those locations is a potential candidate for a new Italian and/or Indian restaurant, at least based on nearby competition.

We will now cluster those locations to create centers of zones containing good locations. Those zones, their centers and addresses will be the final result of our analysis.

In [154]:
from sklearn.cluster import KMeans

number_of_clusters = 10

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_jaude = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_jaude)

folium.Circle(roi_center, radius=1000, color='white', fill=True, fill_opacity=0.4).add_to(map_jaude)
folium.Marker([latitude,longitude],popup='a').add_to(map_jaude)
for lon, lat in cluster_centers:
   # folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_jaude)
    folium.Marker([lat,lon]).add_to(map_jaude)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=4, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_jaude)

map_jaude

In [155]:
for lat,lon in cluster_centers:
    print(lat,lon)

3.084289219640221 45.78232432764761
3.0760329630734358 45.78103098226587
3.09017313632281 45.77478483262186
3.0773236575946967 45.78414804612011
3.0747424125958562 45.777913902034506
3.0802014195044296 45.77245381597064
3.081183025604659 45.784145369172954
3.074750745291079 45.78414975833041
3.0876128623809613 45.78258165497122
3.07731503808972 45.7779121901945


The clusters generated are groupings of our candidate locations and are placed in the zones 'rich' with candidate addresses.<br>
Addresses of these clusters would be a good starting point to explore the neighbourhood areas further to find the best possible location.

We need to **reverse the geocode** of the candidate ares to get the addresses which can be presented in our report to stakeholders.

In [163]:
from geopy.distance import geodesic
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')

for lat, lon in cluster_centers:
    geolocator = Nominatim(user_agent="fr_explorer")
    location = geolocator.reverse([lon,lat])
    #location = location.replace(', France', '') #  remove country part of the address
    if location is None:
        location = 'NO ADDRESS'    
    candidate_area_addresses.append(location)
    cl_d=[lat,lon]
    cc = [longitude,latitude]
    d= geodesic(cl_d, cc).kilometers
    #print(d)
    print('{}{} => {:.2f}km from Place de Jaude'.format(location, ' '*(30-len(location)), d))
    

Addresses of centers of areas recommended for further analysis

16, Rue Bergier, Ecoquartier de Tremonteix, Saint-Alyre, Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, 63000, France                             => 0.70km from Place de Jaude
Fontgiève, Rue Pierre Besset, Ecoquartier de Tremonteix, Fontgiève, Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, 63000, France                             => 0.85km from Place de Jaude
Hélicoptère Gazelle, Cours Sablon, Saint-Jacques, Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, 63000, France                             => 0.91km from Place de Jaude
Central téléphonique, Boulevard Lavoisier, Ecoquartier de Tremonteix, Fontgiève, Clermont-Ferrand, Puy-de-Dôme, Auvergne-Rhône-Alpes, France métropolitaine, 63000, France                             => 1.01km from Place de Jaude
Lycée professionnel Amédée Gasquet, Rue Antoine Menat, Ecoquartier de Tremonteix, Bonnab

This concludes our analysis. We have created 10 addresses representing centers of zones containing locations with low number of restaurants and no Italian and/or Indian restaurants nearby, all zones being fairly close to city center (all less than 1km from Place de Jaude). 

These addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Bonnbaud and Fontgieve areas, which we have identified as interesting due to being a predominatly residential area with schools and gym and parks, fairly close to city center and well connected by public transport.

In [164]:
map_jaude = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(roi_center,popup='a').add_to(map_jaude)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]]).add_to(map_jaude) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=4, color='green', fill=True, fill_color='green', fill_opacity=1).add_to(map_jaude)
map_jaude

## Result and Discussion

Our analysis shows that there are quite a number of restaurants near the city center, about 52 restaurants. Our area of analysis was 1 km from the city center. We also have areas of low restaurants density that are close to the city center. These are in the area of west of the city center.<br>
So our efforts were to identify areas close to the city center but with a lesser density of restaurants.<br>
Our HeatMap of the restaurants in the area shows high concentration of restaurants in the North, North east and south east of the city center

The HeatMap of Italian restaurants is not very 'hot' and in case of the Indian restaurant category we see that the map is pretty cool. 
There is no visible density of restaurants close to the city center and also within 1 km from the city centre. There is just 1 restaurant in the Fontgieve quarters which is about 700 meters from the city centre.<br>
The total presence of Italian restaurants is only 9% and for India restaurants it is just 1% of the total number of restaurants.<br>

Taking into account the specification of having not more than 3 restaurants in the radius of 25 mts and no Italian/Indian restaurant in the vicinity of 300 mts, we put focus to the Bonnabaud quartier of the city. This area is not far from the city center. It is about 500 meters from Place de Jaude. <br>
We also have a low density of restaurants in this area. 

Analysis of the map created and also of websites relating to the Bonnabaud areas shows us that Rue Blatin and Avenue Julien are two of the main streets in this area leading to the cut center. These streets are lined with residences as well as shopping areas and flagship shops leading to the city centre.<br>

We then defined new grid location area which would be around Bonnabaud. For the new ares we had the neighbourhoods closer to each other at 50mts apart and 500 meters from the city centre.

The candidate locations were then clustered to create centers of zones containing good locations. <br>
The result of all this is that we have 10 addresses having our candidate areas to explore further. These 10 addresses should be taken only as the starting point for more detailed analysis. This could mean analysis on various factors like street-level exploration, area, cost etc.
The cluster zones as well as the candidate areas generated are feasible for having and Indian and/or Italian restaurant. The final decision would be upto the stakeholders based on various external factors and considerations.


## Conclusion

Purpose of this project was to identify  areas close to city center of Clermont-Ferrand with low number of restaurants (particularly Italian and Indian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian and/or Indian restaurant. <br>
By calculating restaurant density distribution from Foursquare data we have first identified areas that justify further analysis , and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.<br>
The cluster zones as well as the candidate areas generated are feasible for having and Indian and/or Italian restaurant. The final decision would be upto the stakeholders based on various external factors and considerations

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.